# Initialize Notebook

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
import ROOT
import uproot
import os
import sys
import tqdm
import pandas as pd

%jsroot on
# %matplotlib qt
%matplotlib inline

Welcome to JupyROOT 6.30/00


In [2]:
os.environ['PATH'] = '/Library/TeX/texbin:' + os.environ['PATH']
mpl.rc('font', **{'family':'serif', 'serif':['Computer Modern Roman']})
params = {'backend': 'pdf',
          'axes.labelsize': 20,
          'font.size': 20,
          'legend.fontsize': 20,
          'xtick.labelsize': 20,
          'ytick.labelsize': 20,
          'text.usetex': True,
          'figure.figsize': (5,5),
          'axes.unicode_minus': True}
mpl.rcParams.update(params)

# Load Data

In [3]:
# file_input_name = '../runs/calibration/calibration_-0.85000_-0.85000_-0.85000.root'
# file_input_name = '../runs/calibration/calibration_-0.85000_-0.85000_-0.85000.root'
# file_input_name = '../runs/output_NEST_e-_1MeV.root'
file_input_name = '../build/event_total.root'
# file_input_name = '../build/event.root'
histogram_directory = 'photoSensor_hits_histograms'
file_input_pyroot = ROOT.TFile(file_input_name)
file_input_uproot = uproot.open(file_input_name)

histogram_names = [name for name in file_input_uproot.keys() if "/photoSensor_" in name]
print(histogram_names)

histogram_titles = [file_input_pyroot.Get(name).GetTitle() for name in histogram_names]
print(histogram_titles)

histogram_positions = [(float(title.split('_')[2]), float(title.split('_')[3]), float(title.split('_')[4])) for title in histogram_titles]
print(histogram_positions)

histogram_wall = [title.split('_')[1] for title in histogram_titles]
print(histogram_wall)

tree_names = [name for name in file_input_uproot.keys() if histogram_directory not in name]
print(tree_names)

['photoSensor_hits_histograms/photoSensor_0;1', 'photoSensor_hits_histograms/photoSensor_1;1', 'photoSensor_hits_histograms/photoSensor_2;1', 'photoSensor_hits_histograms/photoSensor_3;1', 'photoSensor_hits_histograms/photoSensor_4;1', 'photoSensor_hits_histograms/photoSensor_5;1', 'photoSensor_hits_histograms/photoSensor_6;1', 'photoSensor_hits_histograms/photoSensor_7;1', 'photoSensor_hits_histograms/photoSensor_8;1', 'photoSensor_hits_histograms/photoSensor_9;1', 'photoSensor_hits_histograms/photoSensor_10;1', 'photoSensor_hits_histograms/photoSensor_11;1', 'photoSensor_hits_histograms/photoSensor_12;1', 'photoSensor_hits_histograms/photoSensor_13;1', 'photoSensor_hits_histograms/photoSensor_14;1', 'photoSensor_hits_histograms/photoSensor_15;1', 'photoSensor_hits_histograms/photoSensor_16;1', 'photoSensor_hits_histograms/photoSensor_17;1', 'photoSensor_hits_histograms/photoSensor_18;1', 'photoSensor_hits_histograms/photoSensor_19;1', 'photoSensor_hits_histograms/photoSensor_20;1', '

In [4]:
tree = file_input_uproot['photoSensor_hits;1']

position_relative_x = tree['photoSensor_hits_position_relative_x'].array()
position_relative_y = tree['photoSensor_hits_position_relative_y'].array()

histogram = file_input_pyroot.Get(histogram_names[0])
n_bins_x = histogram.GetNbinsX()
n_bins_y = histogram.GetNbinsY()
values = np.zeros((n_bins_x, n_bins_y))
x_edges = []
y_edges = []
for i in range(1, n_bins_x + 2):
    x_edges.append(histogram.GetXaxis().GetBinLowEdge(i))
for i in range(1, n_bins_y + 2):
    y_edges.append(histogram.GetYaxis().GetBinLowEdge(i))
x_edges[ 0] -= 1e-9
y_edges[ 0] -= 1e-9
x_edges[-1] += 1e-9
y_edges[-1] += 1e-9

position_relative_x_bins = pd.Series(pd.cut(np.array(position_relative_x), bins=x_edges, retbins=False))
position_relative_y_bins = pd.Series(pd.cut(np.array(position_relative_y), bins=y_edges, retbins=False))
position_relative_x_binned = position_relative_x_bins.apply(lambda x: (x.right + x.left) / 2)
position_relative_y_binned = position_relative_y_bins.apply(lambda x: (x.right + x.left) / 2)
position_relative_x_nBin = position_relative_x_bins.cat.codes
position_relative_y_nBin = position_relative_y_bins.cat.codes

position_initial_x = tree['photoSensor_hits_position_initial_x'].array()
position_initial_y = tree['photoSensor_hits_position_initial_y'].array()
position_initial_z = tree['photoSensor_hits_position_initial_z'].array()

time = tree['photoSensor_hits_time'].array()
photoSensorID = tree['photoSensor_hits_photoSensorID'].array()
photoSensorPosition = [(float(ID.split('_')[2]), float(ID.split('_')[3]), float(ID.split('_')[4])) for ID in photoSensorID]
photoSensorWall = [ID.split('_')[1] for ID in photoSensorID]
photoSensorDirection = []
for wall in photoSensorWall:
    if wall == '+x':
        photoSensorDirection.append([-1, 0, 0])
    elif wall == '-x':
        photoSensorDirection.append([+1, 0, 0])
    elif wall == '+y':
        photoSensorDirection.append([0, -1, 0])
    elif wall == '-y':
        photoSensorDirection.append([0, +1, 0])
    elif wall == '+z':
        photoSensorDirection.append([0, 0, -1])
    elif wall == '-z':
        photoSensorDirection.append([0, 0, +1])

df_hits = pd.DataFrame(columns=['sensor_name', 'sensor_direction', 'sensor_position', 'relativePosition', 'relativePosition_binned', 'time'])
df_hits['sensor_name'] = photoSensorID
df_hits['sensor_direction'] = photoSensorDirection
df_hits['sensor_position'] = photoSensorPosition
df_hits['relativePosition'] = list(zip(position_relative_x, position_relative_y))
df_hits['relativePosition_binned'] = list(zip(position_relative_x_binned, position_relative_y_binned))
df_hits['relativePosition_nBin'] = list(zip(position_relative_x_nBin, position_relative_y_nBin))
df_hits['initialPosition'] = list(zip(position_initial_x, position_initial_y, position_initial_z))
df_hits['time'] = time

# print(df_hits)

In [5]:
primary_tree = file_input_uproot['primary;1']
position_x = primary_tree['primary_position_x'].array()
position_y = primary_tree['primary_position_y'].array()
position_z = primary_tree['primary_position_z'].array()
time = primary_tree['primary_time'].array()

df_primary = pd.DataFrame(columns=['position', 'time'])
df_primary['position'] = list(zip(position_x, position_y, position_z))
df_primary['time'] = time

# Constants

In [6]:
CM_PER_RAD = 34.14230382
MM_PER_CM = 10
CM_PER_M = 100
Y_LIM = 10 * MM_PER_CM
C_M = 299792458
C_CM = C_M * CM_PER_M
N_XENON = 1.84
C_XENON_M_PER_S = C_M / N_XENON
C_XENON_CM_PER_S = C_XENON_M_PER_S * CM_PER_M
C_XENON_MM_PER_S = C_XENON_CM_PER_S * MM_PER_CM
NS_PER_S = 1e9
C_XENON_MM_PER_NS = C_XENON_MM_PER_S / NS_PER_S

# Define Reconstruction Methods

In [7]:
def make_RThetaPhi(df_hits):
    def rToTheta(r):
        return r/(CM_PER_RAD*MM_PER_CM)
        
    r_list = []
    theta_list = []
    phi_list = []

    for index, row in tqdm.tqdm(df_hits.iterrows(), total=len(df_hits)):
        x, y = row['relativePosition']
        
        r = np.sqrt(x**2 + y**2)
        if r > Y_LIM:
            df_hits.drop(index, inplace=True)
            continue

        theta = rToTheta(r)
        phi = np.arctan(y / x)
        
        r_list.append(r)
        theta_list.append(theta)
        phi_list.append(phi)

    df_hits['r'] = r_list
    df_hits['theta'] = theta_list
    df_hits['phi'] = phi_list

    df_hits.reset_index(drop=True, inplace=True)

In [8]:
def make_reconstructedVector_direction(df_hits):
    def get_rotationMatrix(vector, target_direction):
        vector = vector / np.linalg.norm(vector)
        target_direction = target_direction / np.linalg.norm(target_direction)

        axis_of_rotation = np.cross(vector, target_direction)

        angle = np.arccos(np.dot(vector, target_direction))

        rotation_matrix = np.array([[np.cos(angle) + axis_of_rotation[0]**2 * (1 - np.cos(angle)),
                                    axis_of_rotation[0] * axis_of_rotation[1] * (1 - np.cos(angle)) - axis_of_rotation[2] * np.sin(angle),
                                    axis_of_rotation[0] * axis_of_rotation[2] * (1 - np.cos(angle)) + axis_of_rotation[1] * np.sin(angle)],
                                    [axis_of_rotation[1] * axis_of_rotation[0] * (1 - np.cos(angle)) + axis_of_rotation[2] * np.sin(angle),
                                    np.cos(angle) + axis_of_rotation[1]**2 * (1 - np.cos(angle)),
                                    axis_of_rotation[1] * axis_of_rotation[2] * (1 - np.cos(angle)) - axis_of_rotation[0] * np.sin(angle)],
                                    [axis_of_rotation[2] * axis_of_rotation[0] * (1 - np.cos(angle)) - axis_of_rotation[1] * np.sin(angle),
                                    axis_of_rotation[2] * axis_of_rotation[1] * (1 - np.cos(angle)) + axis_of_rotation[0] * np.sin(angle),
                                    np.cos(angle) + axis_of_rotation[2]**2 * (1 - np.cos(angle))]])
        return rotation_matrix

    output_vectors = []
    for _, row in df_hits.iterrows():
        theta = row['theta']
        phi = row['phi']
        sensor_direction = row['sensor_direction']
        
        output_vector = [np.sin(theta) * np.cos(phi), np.sin(theta) * np.sin(phi), np.cos(theta)]
        
        rotationMatrix = get_rotationMatrix([0, 0, 1], sensor_direction)
        rotated_vector = np.dot(rotationMatrix, output_vector)
        
        output_vectors.append(rotated_vector)

    df_hits['reconstructedVector_direction'] = output_vectors

In [9]:
def make_reconstructedPoint_primary_distance(df_hits, df_primary):
    closestPoints = np.full((df_hits.shape[0], 3), -1)
    minDistances = np.full((df_hits.shape[0], 1), -1)
    positions = np.array(df_primary['position'].tolist())
    for i, hit_row in tqdm.tqdm(df_hits.iterrows(), total=df_hits.shape[0]):
        recoVectorDirection = hit_row['reconstructedVector_direction']
        sensorPosition = hit_row['sensor_position']

        vectors = positions - sensorPosition

        dot_products = np.dot(vectors, recoVectorDirection)
        projections = dot_products / np.linalg.norm(recoVectorDirection)
        closest_points_on_line = sensorPosition + projections[:, np.newaxis] * recoVectorDirection
        distances = np.linalg.norm(closest_points_on_line - positions, axis=1)

        minDistanceIndex = np.argmin(distances)
        closestPoints[i] = closest_points_on_line[minDistanceIndex]
        minDistances[i] = distances[minDistanceIndex]

    df_hits['reconstructedPoint_primary_distance'] = closestPoints.tolist()
    df_hits['reconstructedPoint_primary_distance_r'] = minDistances.reshape(-1).tolist()

In [10]:
def make_reconstructedPoint_initial_distance(df_hits):
    recoVectorDirections = df_hits['reconstructedVector_direction'].values
    sensorPositions = df_hits['sensor_position'].values
    initialPositions = df_hits['initialPosition'].values

    vectors = np.array([np.array(ip) - np.array(sp) for ip, sp in zip(initialPositions, sensorPositions)])

    dot_products = np.array([np.dot(v, rv) for v, rv in zip(vectors, recoVectorDirections)])
    projections = np.array([ dp / np.linalg.norm(rv) for dp, rv in zip(dot_products, recoVectorDirections)])    
    closest_points_on_line = np.array([sp + p * rvd for sp, p, rvd in zip(sensorPositions, projections, recoVectorDirections)])
    distances = np.linalg.norm(np.array([cpol - ip for cpol, ip in zip(closest_points_on_line, initialPositions)]), axis=1)

    df_hits['reconstructedPoint_initial_distance'] = closest_points_on_line.tolist()
    df_hits['reconstructedPoint_initial_distance_r'] = distances.tolist()

# Get Results

In [11]:
search_radii = range(0, 10)
search_numbers = range(0, 10)

df_hits_original = df_hits.copy()

for search_radius in search_radii:
    for search_number in search_numbers:
        df_hits = df_hits_original.copy()
        df_hits['closeHits_count'] = 0
        relative_positions = np.vstack(df_hits['relativePosition_nBin'].values)
        photosensor_ids = df_hits['sensor_name'].values

        for index, row in tqdm.tqdm(df_hits.iterrows(), total=len(df_hits)):
            x, y = row['relativePosition_nBin']
            photosensor_id = row['sensor_name']

            same_photosensor = (photosensor_ids == photosensor_id)
            
            distances = np.max(np.abs(relative_positions[same_photosensor] - np.array([x, y])), axis=1)
            df_hits.at[index, 'closeHits_count'] = np.sum(distances <= search_radius) - 1

        df_hits = df_hits[df_hits['closeHits_count'] >= search_number].copy()

        make_RThetaPhi(df_hits)
        make_reconstructedVector_direction(df_hits)
        make_reconstructedPoint_primary_distance(df_hits, df_primary)
        make_reconstructedPoint_initial_distance(df_hits)

        hist, bins, _ = plt.hist(df_hits['reconstructedPoint_primary_distance_r'], bins=100, alpha=0.5, label='Histogram')
        cumulative_sum = np.cumsum(hist)
        PDF = cumulative_sum/cumulative_sum[-1]
        scaled_hist = hist / cumulative_sum[-1]
        plt.clf()

        # Add vertical line at 1 sigma
        primary_sigma_1 = bins[np.argmin(np.abs(PDF - 0.6827))]
        primary_sigma_2 = bins[np.argmin(np.abs(PDF - 0.9545))]
        primary_sigma_3 = bins[np.argmin(np.abs(PDF - 0.9973))]
        primary_mean = np.mean(df_hits['reconstructedPoint_primary_distance_r'])
        primary_std = np.std(df_hits['reconstructedPoint_primary_distance_r'])

        hist, bins, _ = plt.hist(df_hits['reconstructedPoint_initial_distance_r'], bins=100, alpha=0.5, label='Histogram')
        cumulative_sum = np.cumsum(hist)
        PDF = cumulative_sum/cumulative_sum[-1]
        scaled_hist = hist / cumulative_sum[-1]
        plt.clf()

        # Add vertical line at 1 sigma
        initial_sigma_1 = bins[np.argmin(np.abs(PDF - 0.6827))]
        initial_sigma_2 = bins[np.argmin(np.abs(PDF - 0.9545))]
        initial_sigma_3 = bins[np.argmin(np.abs(PDF - 0.9973))]
        initial_mean = np.mean(df_hits['reconstructedPoint_initial_distance_r'])
        initial_std = np.std(df_hits['reconstructedPoint_initial_distance_r'])

        print('[{:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}, {:.2f}]'.format(search_radius, search_number, primary_mean, primary_std, primary_sigma_1, primary_sigma_2, primary_sigma_3, initial_mean, initial_std, initial_sigma_1, initial_sigma_2, initial_sigma_3))

100%|██████████| 42933/42933 [02:39<00:00, 269.74it/s]


[0.00, 0.00, 77.89, 143.02, 40.86, 367.74, 994.26, 360.43, 527.48, 213.03, 1630.99, 2659.01]


100%|██████████| 10654/10654 [00:34<00:00, 308.91it/s]


[0.00, 1.00, 26.91, 31.68, 24.64, 61.60, 234.08, 112.44, 119.27, 100.16, 258.99, 973.73]


100%|██████████| 4067/4067 [00:12<00:00, 329.00it/s]


...

100%|██████████| 36446/36446 [14:51<00:00, 40.88it/s] 


[4.00, 1.00, 43.61, 69.15, 21.54, 172.32, 527.73, 208.41, 292.81, 118.00, 853.37, 1941.72]


 72%|███████▏  | 24082/33353 [02:51<01:06, 140.07it/s]


KeyboardInterrupt: 

<Figure size 500x500 with 0 Axes>